# TP ChromaDB
    
Ce notebook guide vos étapes pour découvrir et pratiquer ChromaDB (base de données vectorielle).
Il est structuré en 4 séquences. Remplissez les cellules TODO.
    
**Prérequis** : Python 3.13+
    
> Astuce : Exécutez chaque cellule dans l’ordre. Documentez vos observations dans les zones prévues.


---
## 0) Préparation de l'environnement

Objectifs :
- Installer les dépendances nécessaires : `chromadb` et un encodeur d'embeddings local (ex : `sentence-transformers`).
- Vérifier les versions.


In [ ]:
# TODO
import chromadb

---
## 1) Découverte et premier index

**Objectifs**
- Comprendre le concept d'embeddings et de recherche de similarité.
- Créer un client ChromaDB, une collection, ajouter quelques documents, interroger.


In [ ]:
# Créez un client avec une base de données persistances
# TODO

In [ ]:
# Créez une collection (<=> table en sgbdr traditionnel)
# TODO

In [ ]:
# Déclarez 3 variables textuelles à ajouter à notre collection
# TODO
doc1 = ""
doc2 = ""
doc3 = ""

In [ ]:
# Déclarez les metadonnées et les identifiants uniques
# TODO
docs = [doc1, doc2, doc3]
ids = []
metadatas = []

In [ ]:
# Ajoutez les données/metadonnées/identifiants à la collection
# TODO

In [ ]:
# Effectuez une recherche de similitude sur une question en rapport avec un des thèmes de vos documents
# TODO

> **Observation (à compléter)**  
- Qu'ai-je observé sur les résultats (ordre, distances, ids) ?  
- À quoi servent les embeddings ?
- Quel est l'embedder par défaut utilisé par chromadb ?


---
## 2) Métadonnées, filtrage, mise à jour et suppression

**Objectifs**
- Ajouter un jeu de documents plus riche (10–15+).
- Utiliser des métadonnées et effectuer des recherches avec filtres.
- Mettre à jour et supprimer des entrées.


In [ ]:
# Créez ou chargez un corpus de 10–15 documents
# Exemple : fabriquez un corpus simple ici. Vous pouvez remplacer par vos propres textes.
# TODO
more_docs = []
more_ids = []
more_metadatas = []

In [ ]:
# Ajouter de nouveaux documents
# TODO

In [ ]:
# Effectuez une recherche sans filtre
# TODO

In [ ]:
# Effectuez une recherche avec filtre métadonnée (ex.: seulement 'technique')
# TODO

In [ ]:
# Effectuez une mise à jour d'un document (update)
# TODO

In [ ]:
# Procédez à une suppression d'un document
# TODO

> **Synthèse (à compléter)**  
- Dans quels cas les filtres par métadonnées m'ont aidé ?  
- Quel impact des mises à jour/suppressions sur les résultats ?


---
## 3) Cas pratique : recherche sémantique sur mini‑corpus

**Objectifs**
- Construire un corpus sur un thème choisi.
- Lancer plusieurs requêtes et analyser la pertinence.
- (Bonus) Chunking si les documents sont longs.


In [ ]:
# Remplacez par votre thème/corpus (ex.: FAQ interne, fiches produits, extraits d'articles)
# TODO:
theme_docs = [
    # Ajoutez ici vos textes liés à un même thème
]
theme_ids = []
theme_metas = []

# Exemple d'ajout (décommentez quand prêts)
# collection_theme = client.get_or_create_collection(name="tp_theme")
# collection_theme.add(documents=theme_docs, ids=theme_ids, metadatas=theme_metas)
# print("Collection 'tp_theme' prête avec", collection_theme.count(), "documents.")

In [ ]:
# TODO: Définissez 3–5 requêtes utilisateur et interrogez la collection
# queries = ["...", "...", "..."]
# for q in queries:
#     res = collection_theme.query(query_texts=[q], n_results=3)
#     print("\nQuery:", q)
#     print(json.dumps(res, indent=2, ensure_ascii=False))

> **Analyse (à compléter)**  
Pour chaque requête :
- Les documents renvoyés sont‑ils pertinents ? Pourquoi ?  
- Les distances/scores reflètent‑ils votre intuition ?


---
## 4) Extension : mini‑application & bonnes pratiques

**Objectifs**
- Construire une petite interface d'interrogation en console.
- Persister les données et discuter des bonnes pratiques.

**Livrable**
- Script/Notebook de mini‑app + fiche « bonnes pratiques » (1 page).


In [ ]:
# Mini‑app console (simple)
COLLECTION_NAME = ""
# Utilise la collection de base. Tapez 'quit' pour sortir.
print("Mini‑app de recherche (collection:", COLLECTION_NAME, ")")
# TODO

---
## Annexes

### Utilisation d'un autre embedder

Par défaut chromadb fait appel à l'embedder `sentence-transformers/all-MiniLM-L6-v2`.  
cf. https://docs.trychroma.com/docs/embeddings/embedding-functions#default-all-minilm-l6-v2

In [ ]:
# Définisez un nouvel embedder local (ex: nomic-embed-text:latest de ollama)
from chromadb.utils import embedding_functions
# TODO

In [ ]:
# Générez les embeddings
# TODO

# Créez une nouvelle collection avec les nouveaux embeddings 
# TODO

In [ ]:
# Alternative avec embedder
# Intégrez la fonction d'embedding lors de la création de la collection
# TODO

In [ ]:
# Testez une requête similaire à la précédente collection
# TODO

### Bonnes pratiques (à discuter / compléter)
- Utiliser `chromadb.PersistentClient()` pour conserver l'index entre sessions.
- Choisir un modèle d'embeddings adapté (local vs API) ; garder la cohérence entre index et requêtes.
- Pré‑traiter les textes (nettoyage, normalisation, segmentation en *chunks* si longs).
- Ajouter des métadonnées utiles dès l’ingestion (source, date, catégorie, langue).
- Tester et itérer : qualifier la pertinence via des jeux de requêtes réels.
- Surveiller le volume de données et la latence ; adapter le backend si besoin.


---
## Références utiles (à consulter)
- Documentation Chroma (Getting Started, API) : https://docs.trychroma.com/
- Répertoire GitHub : https://github.com/chroma-core/chroma
- Concepts embeddings (Sentence-Transformers) : https://www.sbert.net/
    
> **Note** : Vérifiez la version de `chromadb` et les éventuels changements d'API.
